In [1]:
import pandas as pd



In [2]:
dailies = pd.read_parquet('../data/mine/daily.parquet')
gl_teams = pd.read_parquet('../data/mine/gl_teams.parquet')


In [3]:
# Find the opening day team-games (e.g., 1st game of regular season)
glt = gl_teams[(gl_teams['game_type']=='RS')&(gl_teams['team_game_number']==1)][['team', 'game_id', 'team_game_number']].rename(columns={'team': 'team_id'})

In [4]:
# Filter the dailies to just include the opening day games
ods = pd.merge(left=dailies, right=glt, on=['game_id', 'team_id'])

In [5]:
ods

,game_id,game_dt,game_ct,appearance_dt,team_id,player_id,slot_ct,seq_ct,home_fl,opponent_id,...,f_rf_gs,f_rf_out,f_rf_tc,f_rf_po,f_rf_a,f_rf_e,f_rf_dp,f_rf_tp,yr,team_game_number
0,ANA199704020,1997-04-02,0,1997-04-02,ANA,alicl001,2,1,True,BOS,...,0,0.0,0.0,0.0,0.0,0.0,0,0.0,1997,1
1,ANA199704020,1997-04-02,0,1997-04-02,ANA,andeg001,6,1,True,BOS,...,0,0.0,0.0,0.0,0.0,0.0,0,0.0,1997,1
2,ANA199704020,1997-04-02,0,1997-04-02,ANA,disag001,9,1,True,BOS,...,0,0.0,0.0,0.0,0.0,0.0,0,0.0,1997,1
3,ANA199704020,1997-04-02,0,1997-04-02,ANA,edmoj001,3,1,True,BOS,...,0,0.0,0.0,0.0,0.0,0.0,0,0.0,1997,1
4,ANA199704020,1997-04-02,0,1997-04-02,ANA,erstd001,1,1,True,BOS,...,0,0.0,0.0,0.0,0.0,0.0,0,0.0,1997,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31817,WS4187205140,1872-05-14,0,1872-05-14,CL1,paboc101,9,1,False,WS4,...,0,0.0,0.0,0.0,0.0,0.0,0,0.0,1872,1
31818,WS4187205140,1872-05-14,0,1872-05-14,CL1,simmj101,7,1,False,WS4,...,1,27.0,0.0,0.0,0.0,0.0,0,0.0,1872,1
31819,WS4187205140,1872-05-14,0,1872-05-14,CL1,sutte101,2,1,False,WS4,...,0,0.0,0.0,0.0,0.0,0.0,0,0.0,1872,1
31820,WS4187205140,1872-05-14,0,1872-05-14,CL1,whitd102,4,1,False,WS4,...,0,0.0,0.0,0.0,0.0,0.0,0,0.0,1872,1


In [19]:
# Games with at least 1 HR
od_hr = ods[ods['b_hr']>0][['yr', 'player_id', 'b_hr']].sort_values('yr')

In [20]:
# Identify streaks by sub
od_hr['streak_id'] = od_hr['yr'] - od_hr.groupby('player_id').cumcount()
od_hr

,yr,player_id,b_hr,streak_id
30306,1872,woodj106,1.0,1872
22065,1874,gednc101,1.0,1874
18929,1904,paref101,1.0,1904
18915,1904,chesj101,1.0,1904
18927,1904,freeb102,1.0,1904
...,...,...,...,...
23758,2019,franm004,1.0,2019
23764,2019,mccua001,1.0,2017
29749,2019,bryak001,1.0,2019
22018,2019,santd002,1.0,2019


In [34]:
grps = od_hr.groupby(['player_id', 'streak_id'])
grps.filter(lambda grp: grp['b_hr'].count()>3)

,yr,player_id,b_hr,streak_id
19411,1955,berry101,1.0,1955
31365,1956,berry101,1.0,1955
19422,1957,berry101,1.0,1955
3177,1958,berry101,1.0,1955
23346,1977,cartg001,1.0,1976
20369,1978,cartg001,1.0,1976
24109,1979,cartg001,1.0,1976
23386,1980,cartg001,1.0,1976
7662,1994,hundt001,1.0,1994
11849,1995,hundt001,1.0,1994


In [35]:
od_hr[od_hr['player_id']=='cartg001']

,yr,player_id,b_hr,streak_id
27969,1975,cartg001,1.0,1975
23346,1977,cartg001,1.0,1976
20369,1978,cartg001,1.0,1976
24109,1979,cartg001,1.0,1976
23386,1980,cartg001,1.0,1976
20461,1985,cartg001,1.0,1980


In [50]:
# most opening day HR
od_hr.groupby('player_id').agg({'b_hr': sum})['b_hr'].nlargest(5)

player_id
dunna001    8.0
grifk002    8.0
robif103    8.0
mathe101    7.0
maysw101    7.0
Name: b_hr, dtype: float64

In [51]:
# most opening days homered on
od_hr.groupby('player_id').agg({'b_hr': len})['b_hr'].nlargest(5)

player_id
robif103    8.0
grifk002    7.0
cartg001    6.0
dunna001    6.0
maysw101    6.0
Name: b_hr, dtype: float64